**Este código foi desenvolvido para extrair os Rasters presentes no Rest Service do Paraná Cidades, sendo Aerolevantamento do ano de 2021 na maioria dos municípios paranaenses, ele irá gerar tiles que deverão ser mosaicados.** 

Para consultar os municípios e as coordenadas, utilize o [link](https://paranainterativo.pr.gov.br/interativo/rest/services/Base_Cartografica/Base_Cartografica_Municipal_Ortofotos/MapServer).


In [1]:
#Preencha em UTM as dimensões do trecho que deseja retirar
start_x= 582827
start_y= 7442412
end_x = 584503
end_y = 7444735
#Insira o Caminho para salvar tiles
caminho= 'C:\Cidade'

In [3]:
import requests
from PIL import Image
from io import BytesIO
import rasterio
from rasterio.transform import from_bounds
import numpy as np

# Gera um quadrado Branco para comparar e retirar as areas vazias
def referencia():
    bbox = '291013,7320704,291213,7320904'
    url = "https://paranainterativo.pr.gov.br/interativo/rest/services/Base_Cartografica/Base_Cartografica_Municipal_Ortofotos/MapServer/export"
    data = {
        "bbox": bbox,
        "f": "image",
        "size": "4096,4096"
    }
    response = requests.post(url, data=data)
    return response.content




def save_image(bbox,ref):
    url = "https://paranainterativo.pr.gov.br/interativo/rest/services/Base_Cartografica/Base_Cartografica_Municipal_Ortofotos/MapServer/export"
    data = {
        "bbox": bbox,
        "f": "image",
        "size": "4096,4096"
    }

    response = requests.post(url, data=data)

    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        if is_same_as_reference_image(response.content,ref):
          print("Branca")
          return False
        image.save(f"image_{bbox}.png")
        print(f'image_{bbox}')
        return True
    else:
        print(f"Erro ao obter imagem para bbox: {bbox}, status code: {response.status_code}"),
        return False

# Função para criar e iterar pelas bboxes
def create_and_tile_bboxes(start_x, start_y, end_x, end_y, step, crs,ref):

    num_images_x = ((end_x - start_x) // step) + 1
    num_images_y = ((end_y - start_y) // step) + 1

    for j in range(num_images_y):
        for i in range(num_images_x):
            bbox = f"{start_x + i * step},{start_y + j * step},{start_x + (i + 1) * step},{start_y + (j + 1) * step}"
            image = save_image(bbox,ref)
            if image:
              print('sucesso')
              png_to_geotiff(f"image_{bbox}.png", f"{caminho}/image_{bbox}.tif", bbox, crs)
            else:
              print('Quadrado Transparente, ignorando')

def png_to_geotiff(png_path, geotiff_path, bbox, crs):
    # Abra a imagem PNG com a biblioteca PIL
    with Image.open(png_path).convert("RGB") as img:
        # Converta a imagem para um array numpy para usar com rasterio
        arr = np.array(img)

    # Divida a string bbox em uma lista de floats
    bbox = list(map(float, bbox.split(',')))

    # Crie a transformação necessária para o arquivo GeoTIFF
    transform = from_bounds(bbox[0], bbox[1], bbox[2], bbox[3], arr.shape[1], arr.shape[0])

    # Escreva o arquivo GeoTIFF
    with rasterio.open(
        geotiff_path,
        "w",
        driver="GTiff",
        height=arr.shape[0],
        width=arr.shape[1],
        count=arr.shape[2],
        dtype=str(arr.dtype),
        crs=crs,
        transform=transform,
    ) as dest:
        dest.write(arr.transpose((2, 0, 1)))


def is_same_as_reference_image(img_content, reference_img_content):
    img = Image.open(BytesIO(img_content)).convert("RGBA")
    reference_img = Image.open(BytesIO(reference_img_content)).convert("RGBA")
    return np.array_equal(np.array(img), np.array(reference_img))


ref = referencia()


# Tamanho do passo para a criação das bboxes
step = 200
crs = "EPSG:31982"
# Chamada da função para fazer o tiling
create_and_tile_bboxes(start_x, start_y, end_x, end_y, step,crs,ref)


image_582827,7442412,583027,7442612
sucesso


RasterioIOError: Attempt to create new tiff file 'D:/Barra do Jacare/image_582827,7442412,583027,7442612.tif' failed: No such file or directory